In [34]:
# --- Olympic Data Cleaning Notebook ---
# This notebook contains all steps to clean the Olympic Bio and Event Result datasets.

import pandas as pd
import numpy as np
from pathlib import Path
import unicodedata
import json

# ============================
# 1. LOAD RAW DATA
# ============================
# Adjust paths to your docker/airflow layout
landing = Path("../data/olympic")

bio = pd.read_csv(landing /"Olympic_Athlete_Bio.csv")
res = pd.read_csv(landing /"Olympic_Athlete_Event_Results.csv")
cou = pd.read_csv(landing /"Olympics_Country.csv")

print("Bio rows:", len(bio))
print("Results rows:", len(res))
print("Results rows:", len(cou))

# ============================
# 2. INITIAL EXPLORATION
# ============================
print("\n--- BIO Missing Values ---")
print(bio.isna().sum())

print("\n--- RESULTS Missing Values ---")
print(res.isna().sum())


print("\n--- COUNTRIES Missing Values ---")
print(cou.isna().sum())


Bio rows: 155861
Results rows: 316834
Results rows: 235

--- BIO Missing Values ---
athlete_id            0
name                  0
sex                   0
born               4053
height            50749
weight            50749
country               0
country_noc           0
description      100998
special_notes     95224
dtype: int64

--- RESULTS Missing Values ---
edition             0
edition_id          0
country_noc         0
sport               0
event               0
result_id           0
athlete             0
athlete_id          0
pos                 0
medal          272147
isTeamSport         0
dtype: int64

--- COUNTRIES Missing Values ---
noc        0
country    0
dtype: int64


In [35]:

# ============================
# 3. CLEANING: ATHLETE BIO
# ============================

# --- ID Cleaning ---
bio['athlete_id'] = pd.to_numeric(bio['athlete_id'], errors='coerce')
bio = bio.drop_duplicates(subset=['athlete_id'], keep='first')
print("Bio rows:", len(bio))

dupes = cou[cou.duplicated(subset=['noc'], keep=False)]
print(dupes)


Bio rows: 155861
     noc                    country
169  ROC                        ROC
234  ROC  Russian Olympic Committee


In [36]:
cou = cou[~((cou['noc'] == 'ROC') & (cou['country'] == 'ROC'))]
dupes = cou[cou.duplicated(subset=['noc'], keep=False)]
print(dupes)

Empty DataFrame
Columns: [noc, country]
Index: []


In [37]:
dupes = cou[cou.duplicated(subset=['country'], keep=False)]
print(dupes)

Empty DataFrame
Columns: [noc, country]
Index: []


In [38]:
cou['country'] = cou['country'].astype(str).str.strip().str.upper()
countries = set(cou['country'].dropna().unique())
bio['country'] = bio['country'].astype(str).str.strip().str.upper()

orphaned = bio[~bio['country'].isin(countries)]
print("\nOrphaned country rows:", len(orphaned))
orphaned.head()



Orphaned country rows: 2544


,athlete_id,name,sex,born,height,weight,country,country_noc,description,special_notes
5,146111,Svetlana Kholomina,Female,9 November 1997,NaN,NaN,ROC,ROC,NaN,NaN
28,24775,Fathi Khorshid,Male,11 December 1937,NaN,NaN,EGYPT UNITED ARAB REPUBLIC,EGY,Growing up in the agricultural town of El Maha...,Listed in Olympians Who Participated in the 19...
49,2821,Ilija Lupulesku,Male,30 October 1967,177.0,79,INDEPENDENT OLYMPIC ATHLETES SERBIA AND MONTE...,IOA,NaN,Husband (divorced) of Jasna Fazlić-Reed Listed...
53,37731,Birte Siech,Female,19 March 1967,180.0,75,EAST GERMANY GERMANY,GDR,Rower Birte Siech won the gold medal with the ...,NaN
103,81801,Lucia Medzihradská,Female,14 November 1968,NaN,NaN,CZECHOSLOVAKIA SLOVAKIA,TCH,NaN,NaN


In [32]:
cou['noc'] = cou['noc'].astype(str).str.strip().str.upper()
country_nocs = set(cou['noc'].dropna().unique())
bio['country_noc'] = bio['country_noc'].astype(str).str.strip().str.upper()

orphaned = bio[~bio['country_noc'].isin(country_nocs)]
print("\nOrphaned country rows:", len(orphaned))
orphaned.head()



Orphaned country rows: 2


,athlete_id,name,sex,born,height,weight,country,country_noc,description,special_notes
9910,97272,Helmuth Schmalzl,Male,8 October 1948,183.0,74,International Federation Representative Italy,IFR,NaN,Cousin of Eberardo Schmalzl
153046,97014,Bernhard Russi,Male,20 August 1948,182.0,71,International Federation Representative Swit...,IFR,Bernhard Russi claims to have taken up skiing ...,Listed in Olympians Involved in James Bond mov...


In [39]:
cou = pd.concat([cou, pd.DataFrame([{'noc': 'IFR', 'country': 'International Federation Representative Italy'}])], ignore_index=True)
cou['noc'] = cou['noc'].astype(str).str.strip().str.upper()
country_nocs = set(cou['noc'].dropna().unique())
bio['country_noc'] = bio['country_noc'].astype(str).str.strip().str.upper()

orphaned = bio[~bio['country_noc'].isin(country_nocs)]
print("\nOrphaned country rows:", len(orphaned))
orphaned.head()


Orphaned country rows: 0


,athlete_id,name,sex,born,height,weight,country,country_noc,description,special_notes


In [40]:
cou['noc'] = cou['noc'].astype(str).str.strip().str.upper()
country_nocs = set(cou['noc'].dropna().unique())
res['country_noc'] = res['country_noc'].astype(str).str.strip().str.upper()

orphaned = res[~res['country_noc'].isin(country_nocs)]
print("\nOrphaned country rows:", len(orphaned))
orphaned.head()



Orphaned country rows: 0


,edition,edition_id,country_noc,sport,event,result_id,athlete,athlete_id,pos,medal,isTeamSport


In [9]:
# --- Name Normalization ---
bio['name'] = bio['name'].astype(str).str.strip()
bio['name_norm'] = bio['name'].apply(lambda s: unicodedata.normalize('NFKC', s))

# --- Sex Normalization ---
bio['sex'] = bio['sex'].astype(str).str.upper().str.strip().replace({'Male':'M','Female':'F'})


# ============================
# 4. CLEANING: EVENT RESULTS
# ============================

# --- Convert numeric IDs ---
res['athlete_id'] = pd.to_numeric(res['athlete_id'], errors='coerce')
res['edition_id'] = pd.to_numeric(res['edition_id'], errors='coerce')

# --- Medal Normalization ---
res['medal'] = res['medal'].astype(str).str.strip()
res['medal'] = res['medal'].replace({'': None, 'nan': None})


# --- Remove Invalid Edition Years ---


# ============================
# 5. RELATIONSHIP CHECKS
# ============================
ath_ids = set(bio['athlete_id'].dropna().astype(int).unique())
res['athlete_id'] = pd.to_numeric(res['athlete_id'], errors='coerce')

orphaned = res[~res['athlete_id'].isin(ath_ids)]
print("\nOrphaned Results Rows:", len(orphaned))




Orphaned Results Rows: 0


In [42]:
# Drop columns from Olympic Bio
bio = bio.drop(columns=['born', 'height', 'weight', 'country', 'description', 'special_notes'])

# Drop columns from Olympic Event Results
res = res.drop(columns=['athlete', 'pos', 'isTeamSport'])

# Check the result
print("Bio columns:", bio.columns)
print("Results columns:", res.columns)


Bio columns: Index(['athlete_id', 'name', 'sex', 'country_noc'], dtype='object')
Results columns: Index(['edition', 'edition_id', 'country_noc', 'sport', 'event', 'result_id',
       'athlete_id', 'medal'],
      dtype='object')


In [43]:
# ============================
# 6. SAVE CLEANED DATA
# ============================
staging = Path("../test/cleaned")
staging.mkdir(parents=True, exist_ok=True)


# Save cleaned datasets as CSV instead of Parquet
bio.to_csv(staging / "Olympic_Athlete_Bio.csv", index=False)
res.to_csv(staging / "Olympic_Athlete_Event_Results.csv", index=False)
cou.to_csv(staging / "Olympics_Country.csv", index=False)


print("\nCleaning completed. Cleaned files saved to staging.")


Cleaning completed. Cleaned files saved to staging.
